In [1]:
import sys
sys.path.append(r"G:\project\map\wsgi\mindmap\oversea")

In [2]:
urls = ['http://cs.mines.edu/CS-Faculty',
            'http://science.iit.edu/computer-science/people/faculty',
            'http://cms.bsu.edu/academics/collegesanddepartments/computerscience/facultyandstaff/faculty',
            'http://www.memphis.edu/cs/people/index.php',
            'http://www.smu.edu/Lyle/Departments/CSE/People/Faculty',
            'https://www.csuohio.edu/engineering/eecs/faculty-staff',
            'http://computerscience.engineering.unt.edu/content/faculty',
            'https://www.odu.edu/compsci/research',
            'http://www.cs.ucf.edu/people/index.php',
            'https://www.binghamton.edu/cs/people/faculty-and-staff.html',
            'http://www.lsu.edu/eng/ece/people/index.php',
        'http://www.uwyo.edu/cosc/cosc-directory/',
        'http://www.cs.siu.edu/faculty-staff/continuing_faculty.php',
        'https://www.uml.edu/Sciences/computer-science/faculty/',
        'http://cs.gsu.edu/role/faculty/',
        'http://www.eecs.ku.edu/people/faculty',
        'http://www.cs.iastate.edu/people',
        'http://www.sc.edu/study/colleges_schools/engineering_and_computing/study/areas_of_study/computer_science_and_engineering/our_people/index.php',
        'http://cs.mst.edu/facultystaffandfacilities/facultydirectory/',
        'https://www.lsu.edu/eng/cse/people/faculty/index.php',
        'http://www.cse.msstate.edu/People/Faculty/faculty-staff.php',
        'http://soc.fit.edu/faculty-and-staff.php',
        'http://www.mtu.edu/cs/department/faculty-staff/',
        'https://www.ccny.cuny.edu/people?cat=All&school=21&dept=68',
        'https://www.cise.ufl.edu/people/faculty',
            ]
examples = ['https://inside.mines.edu/CS-Faculty-and-Staff/TracyCamp',
            'http://science.iit.edu/people/faculty/eunice-santos',
            'http://cms.bsu.edu/academics/collegesanddepartments/computerscience/facultyandstaff/faculty/buispaul',
            'http://www.memphis.edu/cs/people/faculty_pages/william-baggett.php',
            'http://www.smu.edu/Lyle/Departments/CSE/People/Faculty/ThorntonMitchell',
            'https://www.csuohio.edu/engineering/charles-k-alexander-professor',
            'http://www.cse.unt.edu/~rakl',
            'https://www.odu.edu/directory/people/a/achernik',
            'http://www.cs.ucf.edu/~bagci',
            'https://www.binghamton.edu/cs/people/kchiu.html',
            'http://www.lsu.edu/eng/ece/people/Faculty/brown.php',
            'http://www.uwyo.edu/cosc/cosc-directory/jlc/index.html',
            'http://www.cs.siu.edu/~abosu',
            'https://www.uml.edu/Sciences/computer-science/faculty/levkowitz-haim.aspx',
            'http://cs.gsu.edu/profile/rafal-angryk/',
            'http://people.ku.edu/~agah/',
            'http://www.cs.iastate.edu/people/pavan-aduri',
            'http://www.sc.edu/study/colleges_schools/engineering_and_computing/faculty-staff/jasonbakos.php',
            'http://cs.mst.edu/facultystaffandfacilities/jiang/',
            'https://www.lsu.edu/eng/cse/people/faculty/karki.php',
            'http://www.cse.msstate.edu/People/Faculty/rikk-anderson.php',
            'http://www.fit.edu/faculty/profiles/profile.php?tracks=wallen',
            'http://www.mtu.edu/cs/department/faculty-staff/faculty/brown/',
            'https://www.ccny.cuny.edu/profiles/michael-anshel',
            'https://www.cise.ufl.edu/people/faculty/lanthony',
            ]

url_check = {'http://cs.mines.edu/CS-Faculty': 15,
             'http://science.iit.edu/computer-science/people/faculty': 52,
             'http://cms.bsu.edu/academics/collegesanddepartments/computerscience/facultyandstaff/faculty': 19,
             'http://www.memphis.edu/cs/people/index.php': 16,
             'http://www.smu.edu/Lyle/Departments/CSE/People/Faculty': 17,
             'https://www.csuohio.edu/engineering/eecs/faculty-staff': 32,
             'http://computerscience.engineering.unt.edu/content/faculty': 34,
             'https://www.odu.edu/compsci/research': 15,
             'http://www.cs.ucf.edu/people/index.php': 54,
             'https://www.binghamton.edu/cs/people/faculty-and-staff.html': 33,
             'http://www.lsu.edu/eng/ece/people/index.php': 34,
             'http://www.uwyo.edu/cosc/cosc-directory/': 11,
             'http://www.cs.siu.edu/faculty-staff/continuing_faculty.php': 10,
             'https://www.uml.edu/Sciences/computer-science/faculty/': 18,
             'http://cs.gsu.edu/role/faculty/': 31,
             'http://www.eecs.ku.edu/people/faculty': 56,
             'http://www.cs.iastate.edu/people': 51,
             'http://www.sc.edu/study/colleges_schools/engineering_and_computing/study/areas_of_study/computer_science_and_engineering/our_people/index.php': 44,
             'http://cs.mst.edu/facultystaffandfacilities/facultydirectory/': 30,
             'https://www.lsu.edu/eng/cse/people/faculty/index.php': 32,
             'http://www.cse.msstate.edu/People/Faculty/faculty-staff.php': 21,
             'http://soc.fit.edu/faculty-and-staff.php': 14,
             'http://www.mtu.edu/cs/department/faculty-staff/': 27,
             'https://www.ccny.cuny.edu/people?cat=All&school=21&dept=68': 10,
             'https://www.cise.ufl.edu/people/faculty': 1,
             
             }

In [18]:

import crawler
reload(crawler)


<module 'crawler' from 'G:\project\map\wsgi\mindmap\oversea\crawler.py'>

In [19]:
def test_single(i):
    directory_url = urls[i]
    example = examples[i]
    print i, directory_url #, example
    c = crawler.ResearchCrawler(directory_url, example)
    count, l = c.crawl_faculty_list(directory_url, example)
    print count, url_check[directory_url]
    #for e in l:
    #    print e.get("class"), e.parent.get("class")
    #assert url_check[directory_url] <= count, " not beyond"
    faculty_list = []
    for e in l[:]:
        print l.index(e), e.get('href')
        # print e.get_text()
        x = c.dive_into_page(e, True)
        
        #print x['name']
        print x['tags']
        # print x['position']
        #return c, e
        #print x['link']
        print x['website']
        # print x['source_name']
        #faculty_list.append(x)
    #return faculty_list, c

crawler.debug_level = " " + " extract"
# faculty_list, c = test_single(12)`
i = len(urls) - 1
t = test_single(i)

24 https://www.cise.ufl.edu/people/faculty
1 1
0 https://www.cise.ufl.edu/people/faculty/kyla
 line [u'', u'Utilizing signal processing to render 3D audio', u'Using 3D \xa0audio to sonify information and environments', u'Discovering interface design techniques for developing virtual auditory environments', u'Using 3D audio to augment assistive technology for persons with visual impairments', u'Enhancing immersion and realness in virtual worlds using 3D audio']
 sentence ''
 sentence 'Utilizing signal processing to render 3D audio'
Utilizing signal processing to render 3D audio
using Utilizing signal processing to render 3D audio
utilizing Utilizing signal processing to render 3D audio
also Utilizing signal processing to render 3D audio
research Utilizing signal processing to render 3D audio
interests? Utilizing signal processing to render 3D audio
issues Utilizing signal processing to render 3D audio
developing Utilizing signal processing to render 3D audio
like Utilizing signal proces

into Using 3D  audio to sonify information , environments
through Using 3D  audio to sonify information , environments
during Using 3D  audio to sonify information , environments
before Using 3D  audio to sonify information , environments
after Using 3D  audio to sonify information , environments
above Using 3D  audio to sonify information , environments
below Using 3D  audio to sonify information , environments
to Using 3D  audio , sonify information , environments
from Using 3D  audio , sonify information , environments
up Using 3D  audio , sonify information , environments
down Using 3D  audio , sonify information , environments
in Using 3D  audio , sonify information , environments
out Using 3D  audio , sonify information , environments
on Using 3D  audio , sonify information , environments
off Using 3D  audio , sonify information , environments
over Using 3D  audio , sonify information , environments
under Using 3D  audio , sonify information , environments
again Using 3D  audio ,

using Using 3D audio to augment assistive technology for persons with visual impairments
exploring Using 3D audio to augment assistive technology for persons with visual impairments
applications Using 3D audio to augment assistive technology for persons with visual impairments
among Using 3D audio to augment assistive technology for persons with visual impairments
currently Using 3D audio to augment assistive technology for persons with visual impairments
studies Using 3D audio to augment assistive technology for persons with visual impairments
solving Using 3D audio to augment assistive technology for persons with visual impairments
includ(ing|e|es) Using 3D audio to augment assistive technology for persons with visual impairments
mine Using 3D audio to augment assistive technology for persons with visual impairments
i Using 3D audio to augment assistive technology for persons with visual impairments
me Using 3D audio to augment assistive technology for persons with visual impairments

In [20]:
import re
x = 'Utilizing signal processing to render 3D audio'

In [23]:
words = ['utilizing', 'to']
line = x
for w in words:
    line = re.sub(r'\b%s\b' % w + '(?i)', ',', line, re.I)
    print line

, signal processing to render 3D audio
, signal processing , render 3D audio


In [ ]:
import re

In [30]:
name = "usf"
index_url = "http://www.usf.edu/"
c = crawler.CollegeCrawler(name, index_url)
crawler.debug_level = " " + " "
l = c.crawl_bfs([index_url])
print len(l)

94
62


In [37]:
x = "[TA/RA positions in PhD program] I am looking for self-motivated, creative and hard-working Ph.D. students. Please feel free to email me your CV if you are interested in my research. Missouri S&T; is classified as: Doctoral Universities - Higher Research Activity and is ranked:  #63 (tie) in Engineering Programs (doctorate) according to the 2017 US news ranking and #112 in Computer Science according to US news CS ranking."
contain_keys(x, t, True)

True

In [35]:
t = ["position available", "available for", "looking for [a-zA-Z- \.,]*students", "Interested PhD", "recruiting", "accepting [\\w\\s]*students"]

contain_keys(x, t, True)

True

In [21]:
import HTMLParser
html_parser = HTMLParser.HTMLParser()
s = 'http://web.cse.msstate.edu/%7Eszhang/'
html_parser.unescape(s)

'http://web.cse.msstate.edu/%7Eszhang/'

In [36]:
contain_keys(x, t, True)

True

In [ ]:
anchors = find_all_anchor(soup)
faculty_page, mail, page_name = c.get_personal_website(anchors, 
        faculty_link, person['name'])


In [ ]:
faculty_page

In [ ]:
page_c, page_soup = c.open_page(faculty_page)

In [ ]:
words = "(%s)" % '|'.join(e for e in c.key_words[u'其他可能的研究兴趣短语'])

result = soup.find_all(string=re.compile(words, re.I))
print len(result)

In [ ]:
node = result[0]

In [ ]:
re.sub(r"(<\w+[^>]*>)", ".", "<strong>", re.M)

In [ ]:
pattern = re.compile(r"(<.+>)")
z = pattern.sub(".", unicode(node).strip(), re.M)

In [ ]:
pattern.sub(".", z,count=100)

In [ ]:
import os
import json
def save_case(faclist, c):
    dir_name = "test_case/" + c.university_name
    if not os.path.isdir(dir_name):
        os.makedirs(dir_name)
    with open(os.path.join(dir_name, "data.json"), "w") as fp:
        json.dump(faclist, fp)
save_case(faculty_list, c)

In [ ]:
x = "I am looking for hardworking, creative, and self-motivated graduate (PhD & Master’s) and undergraduate students with interests in artificial intelligence, machine learning, and robotics."

print re.search("looking for [\\w\\s\\.-]*students", x)

In [ ]:
crawler.debug_level = 1
for i in range(11, 12):#len(urls)):
    directory_url = urls[i]
    example = examples[i]
    print i, directory_url #, example
    count, l = crawler.crawl_faculty_list(directory_url, example)
    print count, url_check[directory_url]
    #for e in l:
        #print e["href"]
    # assert url_check[directory_url] <= count, " not beyond"
    for e in l[:5]:
        print l.index(e), e.get('href')
        # print e.get_text()
        x = c.dive_into_page(e)

        
        # print x['name']
        # print x['tags']
        print x['website']
        # print x['position']

In [ ]:
for w in x.split():
    if w.isupper():
        print w
    else:
        print w.lower()

In [ ]:
import os
os.makedirs

In [ ]:
import re
re.search("(position available|looking for [\w\s]*students)", x).group(1)

In [ ]:

crawler.debug_level = 0
faculty_link = 'http://www.cs.ucf.edu/people/index.php'
content, soup = c.open_page(faculty_link)
# c.get_research_interests(soup, content, [], faculty_link)
node = soup.find(string=re.compile("Ulas Bagci"))
print node, node.parent.name, node.parent.get('class')
# c.get_open_position(soup, content)

In [ ]:
a = soup.find('a', href='http://www.cs.ucf.edu/~bagci')
la = a.find_parents()[::-1]

In [ ]:
lb = node.find_parents()[::-1]

In [ ]:
import cProfile
crawler = ResearchCrawler()
for url in urls[8:]:
    print url

    result = crawler.crawl_from_directory(url, examples[urls.index(url)])
    print len(result)
    print result[0]

In [ ]:

import pstats  
p = pstats.Stats('prof.log')

In [ ]:
p.sort_stats('cumtime')
p.print_stats()